In [24]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os
import pandas as pd

def biner(o_bin,m_bin,f_bin,f_step,l_step):
    a=np.arange(o_bin,m_bin,f_step)
    b=np.arange(m_bin,f_bin+l_step,l_step)
    return np.concatenate((a,b))

def normalization(hist_list,first_binning):
    for hist in hist_list:
        for i in range(0,len(rebining)):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=1
            if hist.GetBinWidth(i)!=first_binning:
                sf=hist.GetBinWidth(i)/first_binning
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)

In [33]:
##### SELECTING GENERATOR FOR SIGNAL SAMPLE
signal_sample="Signal.root"
production=True

#### CREATING THE CORRESPONDING MC.ROOT FILE EVERYTIME YOU CHANGE THE SIGNAL SAMPLE

os.system("rm MC.root")
os.system("hadd MC.root "+"BG.root "+signal_sample)

0

hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal.root
hadd Target path: MC.root:/


In [34]:
histos_no_rebin={
"delta_phi":[],
"delta_y":[],
"n_bjets":[],
"lepiso":[],
"n_jets_interval":[],

}
histos_rebin={
"lep1_pt":[120,5,10],
"lep2_pt":[120,5,10],
"ljet0_pt":[120,5,10],
"ljet1_pt":[120,5,10],
"pt_bal":[0.3,0.02,0.02],
"Z_centrality":[2,0.1,0.5],
"inv_mass":[120,5,10],
"mass_jj":[1500,250,500],
}
histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)
    
    
for a in histos:
    print(a)

    Signal=ROOT.TFile.Open(signal_sample,"READ")

    signal=Signal.Get(a)
    
    print(signal.Integral())
    
    signal.SetDirectory(0)

    Signal.Close()
   
    hs = ROOT.THStack("hs","")
 
    hs.Add(signal)

    signal.SetFillColor(ROOT.kAzure+8)
 

    canvas3 = ROOT.TCanvas("canvas3")
    canvas3.cd()
    
    ##### DRAWING TOP PAD, SETTING MARGINS #######

    hs.Draw("HIST same")
    
    canvas3.Print(a+".pdf")


delta_phi
0.0
delta_y
0.0
n_bjets
0.0
lepiso
0.0
n_jets_interval
0.0
lep1_pt
0.0
lep2_pt
0.0
ljet0_pt
0.0
ljet1_pt
0.0
pt_bal
0.0
Z_centrality
0.0
inv_mass
0.0
mass_jj
0.0


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file delta_phi.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file delta_y.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file n_bjets.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lepiso.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file n_jets_interval.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep1_pt.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep2_pt.pdf has been created
Warning in <TCanva

### 

### PLOT SUM OF TWO HISTOGRAMS

In [3]:
ratio = ROOT.TH1F( 'total', 'This is the total distribution', 200, 0, 200 )
hs_mc = ROOT.TH1F("hs_mc",'This is the total distribution', 200, 0, 200)
hs_data = ROOT.TH1F("hs_data",'This is the total distribution', 200, 0, 200)
sum_histos={"lep1_pt_topo_dphi_btag_iso_pt2_mass":[120,1,10],"lep2_pt_topo_dphi_btag_iso_pt1_mass":[120,1,10]}
title='pt1+pt2'
for a in sum_histos:
    MC=ROOT.TFile.Open("MC.root","READ")
    Data = ROOT.TFile.Open("Data.root","READ")

    gStyle.SetOptStat(1111111)
    mc=MC.Get(a)
    data=Data.Get(a)

    mc.SetDirectory(0)
    data.SetDirectory(0)

    MC.Close()
    Data.Close()

    hs_mc.Add(mc)
    hs_data.Add(data)
    

ratio.Divide(hs_mc,hs_data)

gStyle.SetOptStat(1111111)

canvas3 = ROOT.TCanvas("canvas3")
canvas3.cd()

pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)

pad1.Draw ()
pad1.cd ()

hs_data.Draw("pe")
hs_mc.Draw("HIST same")

hs_data . GetXaxis (). SetRangeUser (0,200)
hs_mc . GetXaxis (). SetRangeUser (0,200)

legend = ROOT . TLegend (0.8 ,0.4 ,0.9 ,0.6)
legend . AddEntry ( hs_mc ,"MC")
legend . AddEntry ( hs_data ," Data ")

legend . SetLineWidth (0)
legend . Draw ()



hs_mc.SetTitle(title)

canvas3.cd()
pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.05 ,1 ,0.25)
pad2.Draw ()
pad2.cd ()
ratio.Draw ("pe")
ratio.SetStats(0)
ratio . GetYaxis (). SetRangeUser (0.5 ,1.5)
ratio . GetXaxis (). SetRangeUser (0 ,200)
ratio . GetYaxis (). SetTitle ("MC/DATA")
ratio . GetYaxis (). SetTitleSize (0.15)
ratio . GetYaxis (). SetTitleOffset (0.25)
line = ROOT . TLine (0 ,1 ,200 ,1)
line . SetLineColor ( ROOT . kBlack )
line . SetLineWidth (2)
line . Draw (" same ")
canvas3.Print(title+".pdf")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file pt1+pt2.pdf has been created
